In [2]:
import bs4 as bs
import requests 
import pandas as pd
import re
import nltk
nltk.download('punkt')
import truecase
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pysentiment2 as ps
from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text import FreqDistVisualizer
from yellowbrick.datasets import load_hobbies
import matplotlib.pyplot as plt
from time import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from lxml import html
import numpy as np
import praw
from praw.models import MoreComments
import datetime
import emoji
from collections import Counter
import quandl
import seaborn as sns
import statsmodels.formula.api as smf

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Miche\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# required to use the reddit API to run the code
reddit_token = ''
quandl.ApiConfig.api_key = ''
reddit_client = ''
app_name = ''
username = ''
password = ''

In [6]:
# naming the api reddit 
reddit = praw.Reddit(client_id = reddit_client, client_secret=reddit_token, 
                     user_agent= app_name, username = username, password=password)

In [7]:
# Using the API to grab date, title, id and url of discussion threads (it only pulls 250 even tho the limit is set to 1000)
posts = []
wsb_subreddit = []
for post in reddit.subreddit('wallstreetbets').search(query="Daily Discussion", limit=1000): # searching for all posts in wsb that contain Daily Discussion
    time = post.created # grabbing the submission time of the post
    d = datetime.datetime.fromtimestamp(time).isoformat() # formatting the time
    d = d.split('T') # formatting the time
    date = pd.to_datetime(d[0]) # assigning the new formatted date to the variable date
    posts.append([date, post.title, post.id,  post.url]) # grabbing post title, id & url and creating a df
df_discussion = pd.DataFrame(posts,columns=['Date','Title','id', 'Url'])
display(df_discussion)

,Date,Title,id,Url
0,2021-04-12,"Daily Discussion Thread for April 12, 2021",mpao84,https://www.reddit.com/r/wallstreetbets/commen...
1,2021-04-08,"Daily Discussion Thread for April 08, 2021",mmog6c,https://www.reddit.com/r/wallstreetbets/commen...
2,2021-04-09,"Daily Discussion Thread for April 09, 2021",mndw8t,https://www.reddit.com/r/wallstreetbets/commen...
3,2021-04-06,"Daily Discussion Thread for April 06, 2021",ml86d3,https://www.reddit.com/r/wallstreetbets/commen...
4,2021-03-24,"Daily Discussion Thread for March 24, 2021",mc1zp9,https://www.reddit.com/r/wallstreetbets/commen...
...,...,...,...,...
245,2020-07-24,"Daily Discussion Thread for July 24, 2020",hwz949,https://www.reddit.com/r/wallstreetbets/commen...
246,2020-06-10,"Daily Discussion Thread for June 10, 2020",h07z0d,https://www.reddit.com/r/wallstreetbets/commen...
247,2020-07-28,"Daily Discussion Thread for July 28, 2020",hzc3ko,https://www.reddit.com/r/wallstreetbets/commen...
248,2020-06-12,"Daily Discussion Thread for June 12, 2020",h7i2lz,https://www.reddit.com/r/wallstreetbets/commen...


In [8]:
# This function uses the id column to grab the top level comments of each post
# The purpose of this function is make sure the comments of each post are pulled separetly 
# Before comments for two posts would come as one 
# Code is referenced from the PRAW documentation https://praw.readthedocs.io/en/latest/tutorials/comments.html
def com(x):
    '''Takes id of a post and pulls comments for each submission in a dataframe.'''
    post=[]
    sub = reddit.submission(id=x) #response for the submission 
    for top_level_comment in sub.comments: # iterating through each comment in comment.forest
        if isinstance(top_level_comment, MoreComments): # Skips the more comments button
            continue
        post.append([top_level_comment.body]) # Pulling the comment body for each comment & appending it
    return post

In [9]:
df_discussion['Body'] = df_discussion['id'].apply(com) # Pulling the comments of each post & placing it into the body column

In [10]:
# This function is used to pull the emojis from the comments of each post and joins it into one string
# Used later to perform sentiment analysis
def extract_emoji(e):
    '''Takes strings and extracts emojis for a dataframe.'''
    emoji_string = ''
    for i in e:
        emojis = emoji.emoji_lis(i) # emoji_lis returns emoji positions in a string
        emoji_join = (''.join([d['emoji'] for d in emojis])) # Joining the emojis
        emoji_string = emoji_string + emoji_join # appending the emojis into one big string
    return emoji_string

In [11]:
df_discussion["Body"] = df_discussion["Body"].apply(str) # converting comments to string to clean
df_discussion['Emoji'] = df_discussion["Body"].apply(extract_emoji) # Extracting emojis 

In [12]:
# Cleaning the comments by removing stop words, unwanted characters & lemmatization 
stop_words = set(stopwords.words('english'))

def preprocess(x):
    '''Takes a string, makes it truecase, removes unwanted characters, stopwords and stems to its root form'''
    # Truecase
    x = truecase.get_true_case(x)
    
    # Remove unwanted characters
    x = re.sub(r"[\W\d]", " ", x)
    
    #Remove stopwards and lemmatize
    x = [WordNetLemmatizer().lemmatize(w) for w in x.split() if w not in stop_words]
    return ' '.join(x)

In [13]:
# Cleaning the comments by removing the \\n characters 
def replace_char(n):
    '''Takes a string value from a df and removes \\n'''
    n = n.replace('\\n', '')
    return n

In [14]:
df_discussion["Body"] = df_discussion["Body"].apply(replace_char) # Replacing the \\n characters
df_discussion['Body_clean'] = df_discussion["Body"].apply(preprocess) # New column with cleaned comments after preprocessing

In [15]:
# List of blacklist words refrenced from https://github.com/RyanElliott10/wsbtickerbot/issues/3
# Added some additional words onto the that list
blacklist_words = [
      "YOLO", "TOS", "CEO", "CFO", "CTO", "DD", "BTFD", "WSB", "OK", "RH",
      "KYS", "FD", "TYS", "US", "USA", "IT", "ATH", "RIP", "BMW", "GDP",
      "OTM", "ATM", "ITM", "IMO", "LOL", "DOJ", "BE", "PR", "PC", "ICE",
      "TYS", "ISIS", "PRAY", "PT", "FBI", "SEC", "GOD", "NOT", "POS", "COD",
      "AYYMD", "FOMO", "TL;DR", "EDIT", "STILL", "LGMA", "WTF", "RAW", "PM",
      "LMAO", "LMFAO", "ROFL", "EZ", "RED", "BEZOS", "TICK", "IS", "DOW"
      "AM", "PM", "LPT", "GOAT", "FL", "CA", "IL", "PDFUA", "MACD", "HQ",
      "OP", "DJIA", "PS", "AH", "TL", "DR", "JAN", "FEB", "JUL", "AUG",
      "SEP", "SEPT", "OCT", "NOV", "DEC", "FDA", "IV", "ER", "IPO", "RISE"
      "IPA", "URL", "MILF", "BUT", "SSN", "FIFA", "USD", "CPU", "AT",
      "GG", "ELON", "CAP", 'OPEN', "RIDE", "PLAY", "ROPE", "ON", "SHIT"
   ]

In [16]:
# This website keeps track of sentiment, comment volume , popular stocks etc on WSB
# Ref : 'https://swaggystocks.com/dashboard/wallstreetbets/ticker-sentiment'
# Copied the HTML code in the website and scraped the ticker symbols of the most popular stocks on WSB listed on this website
# This part is done manually b/c  scrapy was not working for some reason
# Html_code needs to be updated if the popular stocks on wsb changes
# We know a formal list of all stock symbols can be found & downloaded but we did not want to use that
# b/c WSB sometimes uses words such as SEC in ref to the exchange not the stock
# When we first ran the code with the full list it picked up stocks that were not popular on WSb
# Using this website tailors the stocks to the ones that are popular on WSB

html_code = '''<div class="stocks-table option-volume-table" style="margin-top: 10px;"><div class="volume-ticker-entry due"><p class="volume-ticker-name">SPY</p><p class="volume-ticker-value">2.44K Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TSLA</p><p class="volume-ticker-value">995 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PLTR</p><p class="volume-ticker-value">793 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GME</p><p class="volume-ticker-value">635 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BB</p><p class="volume-ticker-value">476 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MVIS</p><p class="volume-ticker-value">446 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AAPL</p><p class="volume-ticker-value">394 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DISCB</p><p class="volume-ticker-value">330 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DISCA</p><p class="volume-ticker-value">260 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VIAC</p><p class="volume-ticker-value">246 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NIO</p><p class="volume-ticker-value">243 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MSFT</p><p class="volume-ticker-value">229 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AMC</p><p class="volume-ticker-value">170 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AMZN</p><p class="volume-ticker-value">147 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AMD</p><p class="volume-ticker-value">142 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CHWY</p><p class="volume-ticker-value">140 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RKT</p><p class="volume-ticker-value">125 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PLUG</p><p class="volume-ticker-value">120 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CLF</p><p class="volume-ticker-value">115 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">APHA</p><p class="volume-ticker-value">110 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SQ</p><p class="volume-ticker-value">110 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">QQQ</p><p class="volume-ticker-value">103 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TLRY</p><p class="volume-ticker-value">91 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CRSR</p><p class="volume-ticker-value">87 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MU</p><p class="volume-ticker-value">86 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DIS</p><p class="volume-ticker-value">84 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ASO</p><p class="volume-ticker-value">71 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LULU</p><p class="volume-ticker-value">69 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FUBO</p><p class="volume-ticker-value">65 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GLD</p><p class="volume-ticker-value">63 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GOEV</p><p class="volume-ticker-value">59 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UWMC</p><p class="volume-ticker-value">59 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ICLN</p><p class="volume-ticker-value">58 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CHPT</p><p class="volume-ticker-value">58 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">OCGN</p><p class="volume-ticker-value">58 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ARKK</p><p class="volume-ticker-value">56 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BA</p><p class="volume-ticker-value">56 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NOK</p><p class="volume-ticker-value">54 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SNDL</p><p class="volume-ticker-value">45 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DKNG</p><p class="volume-ticker-value">45 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MT</p><p class="volume-ticker-value">44 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NFLX</p><p class="volume-ticker-value">42 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RIOT</p><p class="volume-ticker-value">40 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">XL</p><p class="volume-ticker-value">39 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AAL</p><p class="volume-ticker-value">39 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MARA</p><p class="volume-ticker-value">39 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">F</p><p class="volume-ticker-value">37 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GOOGL</p><p class="volume-ticker-value">36 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FB</p><p class="volume-ticker-value">35 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UVXY</p><p class="volume-ticker-value">35 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SQQQ</p><p class="volume-ticker-value">31 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BABA</p><p class="volume-ticker-value">31 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BIDU</p><p class="volume-ticker-value">30 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SNAP</p><p class="volume-ticker-value">29 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">QS</p><p class="volume-ticker-value">28 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RBLX</p><p class="volume-ticker-value">27 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AMAT</p><p class="volume-ticker-value">26 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TSM</p><p class="volume-ticker-value">26 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NET</p><p class="volume-ticker-value">26 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SOXL</p><p class="volume-ticker-value">24 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MSOS</p><p class="volume-ticker-value">24 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ROPE</p><p class="volume-ticker-value">24 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AMRS</p><p class="volume-ticker-value">24 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CS</p><p class="volume-ticker-value">24 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">INTC</p><p class="volume-ticker-value">24 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NVDA</p><p class="volume-ticker-value">24 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NKLA</p><p class="volume-ticker-value">22 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ZM</p><p class="volume-ticker-value">21 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SLV</p><p class="volume-ticker-value">21 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ROOT</p><p class="volume-ticker-value">21 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PAVE</p><p class="volume-ticker-value">20 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BLNK</p><p class="volume-ticker-value">20 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TLT</p><p class="volume-ticker-value">18 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SPOT</p><p class="volume-ticker-value">17 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TQQQ</p><p class="volume-ticker-value">17 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FCEL</p><p class="volume-ticker-value">17 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GE</p><p class="volume-ticker-value">17 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GM</p><p class="volume-ticker-value">16 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WMT</p><p class="volume-ticker-value">16 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PYPL</p><p class="volume-ticker-value">16 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">XOM</p><p class="volume-ticker-value">16 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BNGO</p><p class="volume-ticker-value">16 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PSTH</p><p class="volume-ticker-value">16 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GNUS</p><p class="volume-ticker-value">15 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GSX</p><p class="volume-ticker-value">15 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PFE</p><p class="volume-ticker-value">15 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">X</p><p class="volume-ticker-value">15 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NUE</p><p class="volume-ticker-value">15 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AR</p><p class="volume-ticker-value">14 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MRNA</p><p class="volume-ticker-value">14 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ZOM</p><p class="volume-ticker-value">14 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FUTU</p><p class="volume-ticker-value">13 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DASH</p><p class="volume-ticker-value">13 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VIACA</p><p class="volume-ticker-value">13 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CMG</p><p class="volume-ticker-value">13 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ROKU</p><p class="volume-ticker-value">13 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GS</p><p class="volume-ticker-value">12 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BMBL</p><p class="volume-ticker-value">12 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HYLN</p><p class="volume-ticker-value">12 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">OGS</p><p class="volume-ticker-value">12 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CX</p><p class="volume-ticker-value">12 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DE</p><p class="volume-ticker-value">11 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DISCK</p><p class="volume-ticker-value">11 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TME</p><p class="volume-ticker-value">11 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BBY</p><p class="volume-ticker-value">11 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ARKG</p><p class="volume-ticker-value">11 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CBS</p><p class="volume-ticker-value">11 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PENN</p><p class="volume-ticker-value">11 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VUZI</p><p class="volume-ticker-value">11 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WKHS</p><p class="volume-ticker-value">11 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HD</p><p class="volume-ticker-value">11 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">IQ</p><p class="volume-ticker-value">10 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MCD</p><p class="volume-ticker-value">10 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PRPL</p><p class="volume-ticker-value">10 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">QQQJ</p><p class="volume-ticker-value">10 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LAC</p><p class="volume-ticker-value">10 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SUMO</p><p class="volume-ticker-value">10 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UPS</p><p class="volume-ticker-value">10 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SBUX</p><p class="volume-ticker-value">10 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PTON</p><p class="volume-ticker-value">10 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TIGR</p><p class="volume-ticker-value">9 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VALE</p><p class="volume-ticker-value">9 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ENPH</p><p class="volume-ticker-value">9 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">XPEV</p><p class="volume-ticker-value">9 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CD</p><p class="volume-ticker-value">9 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UBS</p><p class="volume-ticker-value">9 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TDOC</p><p class="volume-ticker-value">8 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LI</p><p class="volume-ticker-value">8 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ARKF</p><p class="volume-ticker-value">8 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">IDEX</p><p class="volume-ticker-value">8 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RIDE</p><p class="volume-ticker-value">8 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GEVO</p><p class="volume-ticker-value">8 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BAC</p><p class="volume-ticker-value">8 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CAT</p><p class="volume-ticker-value">8 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PINS</p><p class="volume-ticker-value">8 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CLNE</p><p class="volume-ticker-value">8 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MS</p><p class="volume-ticker-value">8 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">EBON</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HYFM</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MP</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ET</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">T</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FAX</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">OPEN</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SEAS</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CCIV</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ETSY</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CO</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LEE</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CCJ</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UAL</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PSLV</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DEN</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HACK</p><p class="volume-ticker-value">7 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">JPM</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">XLF</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UBER</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">COUR</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RC</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PRNT</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TGT</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TBF</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">EAR</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CRSP</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FSR</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SI</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MMM</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SKLZ</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ACB</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DDS</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">EEENF</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GPS</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LE</p><p class="volume-ticker-value">6 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CRM</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BIL</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">JNUG</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UCO</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GSAT</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HTZ</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WOOF</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WFC</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LAD</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CORN</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NTLA</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SPCE</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NNDM</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GRWG</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">XLE</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VLDR</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">USO</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AG</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CCL</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VZ</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">JD</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ELY</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">REV</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VOO</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SNOW</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ABT</p><p class="volume-ticker-value">5 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SENS</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DECK</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WBA</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DDOG</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VIPS</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CSIQ</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">STMP</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DIAL</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GUSH</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">STPK</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TWTR</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MJ</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SP</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LUNG</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CVS</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CRWD</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SILJ</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UI</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CVNA</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ABCL</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RIG</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PLAY</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HOG</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WWE</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">COST</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SOLO</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ARVL</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LUV</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ADP</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RICK</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ON</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ZNGA</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ARKQ</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AZZ</p><p class="volume-ticker-value">4 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SIRI</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PLL</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TM</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">EE</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NI</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LAZR</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LRCX</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">IWM</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DAN</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">COP</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">JMIA</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ALLY</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FSLY</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GOOG</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">XBI</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VZIO</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GPRO</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NKE</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CHEF</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BUG</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WEN</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WIFI</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HGEN</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MLM</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BTO</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TX</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NNOX</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BBD</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">JNJ</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ES</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MGM</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WIRE</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DBX</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DAL</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AIV</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TAN</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PETS</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">KBR</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NAKD</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">KMX</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">V</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LMND</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ZION</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SAND</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HIMS</p><p class="volume-ticker-value">3 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UNP</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FANG</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SNE</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BEPC</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MAXR</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FTCH</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">COO</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NVAX</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">XONE</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GAL</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FDX</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VET</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BEP</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TI</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SPLK</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PBW</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TRMB</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">XLI</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WING</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TU</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AZO</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MRO</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">JCI</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UWM</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ULTA</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TTWO</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TVIX</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RYCEY</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">Z</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">EEM</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BUTT</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TRS</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RIO</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">KL</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CMPS</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WY</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TTD</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">KAR</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UPTY</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MX</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">QQQM</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NLY</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CATH</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">KHC</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RDFN</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CUB</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FUV</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">KTOS</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">COHU</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PS</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TSN</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GILD</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PCG</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TIVO</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ZG</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NSA</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NOVA</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">IPOE</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ING</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SCHN</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GRMN</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DOCU</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FREQ</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CRON</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ARKX</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">COMM</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TLS</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TAIL</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VFF</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TILE</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BBC</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TSE</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FCX</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PEN</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CLOU</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">EB</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UMC</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BCRX</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UCLA</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TCCNF</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VTI</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MDMP</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">JUSHF</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">KRE</p><p class="volume-ticker-value">2 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SURF</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">NEK</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CVX</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">COPX</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">IDT</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ACAD</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LCID</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DGLY</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ORCL</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VER</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">IBM</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SHMP</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">KAI</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">OKE</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BALL</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TGIF</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">IEF</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AOL</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SHIT</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">EA</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SIG</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CRI</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SOS</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DOMO</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">EC</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HSBC</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WPRT</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MNST</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SPWR</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ADT</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CPL</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DNMR</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SNEK</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">USCR</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PLOW</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TJX</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AI</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SDGR</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GD</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PBR</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">JCP</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SFM</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SLY</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">YUM</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AVA</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">INN</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PCVX</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BW</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ETN</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">IWO</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BILI</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ABNB</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GIB</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WIT</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SPXU</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UGL</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ASML</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DISC</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AOR</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ACH</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">KMI</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PHO</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SPT</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ERJ</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TMO</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CHGG</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">KR</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BODY</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TTCF</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VGAC</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UNH</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RES</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ROK</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FI</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">COW</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VMW</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BDRBF</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">WTI</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CIG</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SFTW</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HIMX</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">LYFT</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HY</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CMI</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DB</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RAIL</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ABC</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PWR</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CONE</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ACTC</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SORE</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HIW</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AU</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FSD</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AB</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">REGN</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">QDEL</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SOYB</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CBAT</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">FNKO</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SSD</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">HUM</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">JCOM</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">XSD</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">XME</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AVGO</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SYKE</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">EBAY</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MC</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AMGN</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">EBS</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">GUHBB</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ARGO</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">RAVN</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ABBV</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CPG</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SHW</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">IIPR</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">OPEC</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VDC</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">DSL</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ZEAL</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">PB</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">BJ</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">UL</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">REX</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">REGI</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TGE</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">ARKW</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">SRPT</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">VUG</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">REM</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">MUI</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">XP</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">CYDY</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">TXRH</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">IDK</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div><div class="volume-ticker-entry due"><p class="volume-ticker-name">AXE</p><p class="volume-ticker-value">1 Mentions</p><p class="volume-ticker-select">SELECT</p></div></div>'''
soup = bs.BeautifulSoup(html_code, "html.parser")
max_page = soup.find_all('p')

In [17]:
# Creating stock list from the tickers found in the html code above
stonks = []
for i in max_page:
    i = i.getText()
    if len(i)<= 4: # Only grabbing words less than/equal to 4 letters(tickers) & excluding other text found
        stonks= stonks + [i]

In [18]:
# Using the stock list above, we loop the each word in the cleaned body column & look for words that appear in the stock list
# We then calculate the # of times each stock is mentioned under each post
def stock_extracter(s):
    '''Takes a string and splits it by space and loops through to find the stocks mentioned under each post
    and the number of times it is mentioned. Uses the stock listed created above.'''
    word = []
    stonks_all = []
    stonks_mentioned = {}
    data = s.split() # Split the cleaned comments into a list
    for i in data: # loop through each word
        if len(i)<= 4: # Filter for words less than/ equal to 4 letters
            i = i.upper() # Make the words uppercase
            if i not in blacklist_words: # if word is not listed in our blacklist words list from above
                    word = word + [i] # We append the words into a list
                    
    for i in word: # Loop through the word list
        if i in stonks: # If the word appears in our stock list
            stonks_all = stonks_all + [i] # append it to a new list
    for t in stonks_all: # Iterate through this new stock list
        if t in stonks_mentioned: # if its in our dict add 1
            stonks_mentioned[t]+=1
        else:
            stonks_mentioned[t]=1 # if not, equal it to 1
    return stonks_mentioned

In [19]:
df_discussion['Stocks Mentioned'] = df_discussion['Body_clean'].apply(stock_extracter) # Applying our stock extractor

In [20]:
# Pulling data from our dictionaries
lm_rd = pd.read_excel('LoughranMcDonald_MasterDictionary_2018.xlsx') #updated with reddit words
ed = pd.read_excel('Emojis_Dictionary.xlsx')

# get the lists of positive/negative words
lm_rdpos = list(lm_rd[lm_rd.Positive==1]['Word'])
lm_rdneg = list(lm_rd[lm_rd.Negative==1]['Word'])

# get the lists of positive/negative emojis
edpos = list(ed[ed.Positive!=0]['Emoji'])
edneg = list(ed[ed.Negative!=0]['Emoji'])

In [21]:
# Calculates text sentiment using the updated LM dictionary
def sent_score(x):
    '''The function takes cleaned text and calculates sentiment score. '''
    nwords = [] # Total words
    nneg = [] # Total neg words
    npos = [] # Total pos words
    x = x.split() # Splitting the body by space
    for r in x: # loop through each word
        r = r.upper() # make each word uppercase
        nwords = nwords +[r] # add word to word count
        if r in lm_rdpos: # check if word is a positive word
            npos = npos + [r] # append
        elif r in lm_rdneg: # check if word is a negative word
            nneg = nneg + [r] # append
    global sent
    sent = ((len(npos)- len(nneg)) / len(nwords)) # Calculate sent score
    score = {'Positive': len(npos), 'Negative':len(nneg), 'Score': sent, 'Word_Count': len(nwords)} # return all values as a dict
    return score

In [22]:
# Calculates text sentiment using the updated emoji dictionary
def sent_emoji(x):
    '''The function takes emojis and calculates sentiment score.'''
    nemojis = [] # Total neg emojis
    pemojis = [] # Total pos emojis
    total_emojis = [] # Total emojis
    x = re.findall(r'[^\w\s,.]', x) # finds all emojis into a list
    for r in x:
        total_emojis = total_emojis + [r] # all emojis
        if r in edpos:
            pemojis = pemojis + [r] # list of positive emojis
        elif r in edneg:
            nemojis = nemojis + [r] # list of negative emojis
    if len(total_emojis) > 0:
        sent = ((len(pemojis)- len(nemojis)) / len(total_emojis)) # calculate sentiment of emojis
    else:
        sent = 0 #exception error - row 246 has no emojis
    score = {'Positive': len(pemojis), 'Negative':len(nemojis), 'Score': sent, 'Emoji_Count': len(total_emojis)} # returns all values in a dict
    return score

In [23]:
df_discussion['Reddit_Sentiment_text'] = df_discussion['Body_clean'].apply(sent_score) # Calculating Text Sent

In [24]:
df_discussion['Reddit_Sentiment_emoji'] = df_discussion['Emoji'].apply(sent_emoji) # Calculating Emoji Sent

In [26]:
# Pulling all the values in the Reddit_Sentiment_text & Reddit_Sentiment_emoji columns & placing them in separate columns
# We realized later we wanted to use a total sentiment score, along with the two separate scores
pos_e = [] # pos emojis
pos_t = [] # pos text
neg_e = [] # neg emojis
neg_t = [] # neg text
word_e = [] # total emojis
word_t = [] # total words
total_pos = [] # total positive (emoji + text)
total_neg = [] # total negative (emoji + text)
net = [] # Total pos - Total neg
tot_w = [] # total words + emojis
total_sent = [] # Total sentiment score
score_e =[] # emoji sent score
score_t = [] # text sent score
for i,z in zip(df_discussion['Reddit_Sentiment_emoji'].values,df_discussion['Reddit_Sentiment_text'].values ):
    pos_e = pos_e + [float(i['Positive'])] # total postive emojis
    pos_t = pos_t + [float(z['Positive'])] # total positive text
    neg_e = neg_e + [float(i['Negative'])] # total neg emojis
    neg_t = neg_t + [float(z['Negative'])] # total neg text
    word_e = word_e + [float(i['Emoji_Count'])] # total emojis
    word_t = word_t + [float(z['Word_Count'])] # total words
for s,y in zip(pos_e,pos_t):
    total_pos = total_pos +[(s+y)] #total pos text + emojis
for o,p in zip(neg_e,neg_t):
    total_neg = total_neg + [(o+p)] # total neg text + emojis
for r,u in zip(total_pos,total_neg):
    net = net + [(r-u)]  # total pos - total neg
for k,p in zip(word_e,word_t):
    tot_w = tot_w +[(k+p)] # total word + emoji
for q,m in zip(net,tot_w):
    total_sent = total_sent + [(q/m)] # total sent = ((total pos text + emojis) - (total neg text + emojis)) / (total word + emoji)
for m in (df_discussion['Reddit_Sentiment_emoji'].values):
    score_e = score_e + [float(m['Score'])] # Pulling all emoji sent scores
for b in (df_discussion['Reddit_Sentiment_text'].values):
    score_t = score_t + [float(b['Score'])] # Pulling all text sent scores

In [27]:
df_discussion['Total_sent'] = total_sent # adding total sent
df_discussion['Emoji_sent'] = score_e # adding emoji sent
df_discussion['Text_sent'] = score_t # adding text sent
df_discussion['Comment Volume'] = tot_w # adding total positive
df_discussion['Total_Positive'] = total_pos # total positive
df_discussion['Total_Negative'] = total_neg # total negative

## Analysis Begins

In [28]:
df_discussion['Date'] = pd.to_datetime(df_discussion['Date'], format='%Y-%m-%d') # Converting to datetime format
df_discussion = df_discussion.set_index('Date') # Indexing date column

In [29]:
# We are taking the values from the stocks mentioned column where it has each stock & how many times the stock was mentioned
# under each post and we are summing them across our sample.
# Ex. there are 3 posts & GME is mentioned once for post 1, twice for post 2, 5 times for post 3
# The function will sum this and show GME was mentioned in total 8 times across 3 posts
def top_3(x):
    '''Takes a dict and sums the total mentions of all stocks across the sample in a df.
    Args:
        x: is any dictonary value.'''
    key_lst = [] # dict key list
    value_lst = [] # dict value list
    dict_1 = {} # New dict
    for key, value in x.items(): # B/c its a df, the key is the index
        for k, v in value.items(): # we need to loop one more time to get the dict values
            key_lst = key_lst + [k] # append keys
            value_lst = value_lst + [v] # append values
            for t,z in zip(key_lst,value_lst): # loop & sum the freq
                if t not in dict_1: # if key not in the list
                    dict_1[t] = z # z is the value (Ex. GME is not in the list and GME has count of 3. z is 3)
                else:
                    dict_1[t] += z  # Otherwise we add z if the key is in the list (GME is already there with a count of 3 & z the count value of GME in another post is 1. Then we add 3+ z(1))           
    return dict_1 # Return the new dict

In [31]:
top_3_stocks = df_discussion.loc[:,['Stocks Mentioned']]
dict_2 = top_3_stocks.apply(top_3) # Applying the counter function to the stocks mentioned column

In [ ]:
# Most mentioned stocks across sample (done manually by sorting dict)
sorted((dict_2.values)[0].values(),reverse=True) #Top 5 mentions 48650896,35518006,28098901,14568748,13771119
key1 = [k for k, v in (dict_2.values)[0].items() if v == 48650896] #GME
key2 = [k for k, v in (dict_2.values)[0].items() if v == 35518006] #PLTR
key3 = [k for k, v in (dict_2.values)[0].items() if v == 28098901] #AMC
key4 = [k for k, v in (dict_2.values)[0].items() if v == 14568748] #BB
key5 = [k for k, v in (dict_2.values)[0].items() if v == 13771119] #TSLA

In [32]:
df_discussion

,Title,id,Url,Body,Emoji,Body_clean,Stocks Mentioned,Reddit_Sentiment_text,Reddit_Sentiment_emoji,Total_sent,Emoji_sent,Text_sent,Comment Volume,Total_Positive,Total_Negative
Date,,,,,,,,,,,,,,,
2021-04-12,"Daily Discussion Thread for April 12, 2021",mpao84,https://www.reddit.com/r/wallstreetbets/commen...,"[[""This is fine, I didn't really want to be fi...",🤔😂😃🚀🤡🤡🤡🤡🙌🚀🤡🤡🤡😤💩🚀🚀🌕🦍💪💎👐🤦♂😭😭🍏🤑🤑🤑🅱🅱💰😭😭😭😂😂🏿🤣🥺🥺🥺🚀🤡🚀...,This fine I really want financially secure any...,"{'SPY': 26, 'PLTR': 58, 'APHA': 48, 'TLRY': 8,...","{'Positive': 40, 'Negative': 48, 'Score': -0.0...","{'Positive': 46, 'Negative': 37, 'Score': 0.08...",0.000244,0.082569,-0.002003,4104.0,86.0,85.0
2021-04-08,"Daily Discussion Thread for April 08, 2021",mmog6c,https://www.reddit.com/r/wallstreetbets/commen...,"[[""If your wife asks where did the money go ju...",😎😎🤡✅🤔😎🤬😀🚀♿😑💥🍎🤤🚀👀🌈💅😞😞😞🤨🐻🥵🥵🚀🚀🚀🚀🚀🚀🤡🚀🚀👀😍😭🤷♂🤔😡🥵🍏😏🚀💎...,If wife asks money go say It long term investm...,"{'SPY': 31, 'PLTR': 38, 'RKT': 3, 'AAPL': 22, ...","{'Positive': 60, 'Negative': 49, 'Score': 0.00...","{'Positive': 26, 'Negative': 15, 'Score': 0.22...",0.004936,0.220000,0.002496,4457.0,86.0,64.0
2021-04-09,"Daily Discussion Thread for April 09, 2021",mndw8t,https://www.reddit.com/r/wallstreetbets/commen...,"[[""Y'all ever be on page 47 of Pornhub tryna l...",🚀🚀🚀🤡🤡🤡😩😩❤😍🍏🚀🚀🚀🌙🤡🤡🤡🚀🚀🚀🏎😤⚽🍏🚀🤡🥴🍏🐻🔫🐮🚀😎🦧🗣🤡🤡🤡👉📉📉😭✋🏻🚀...,Y ever page Pornhub Tryna look one Video think...,"{'ASO': 19, 'SPY': 47, 'NET': 1, 'REGI': 1, 'P...","{'Positive': 56, 'Negative': 42, 'Score': 0.00...","{'Positive': 76, 'Negative': 23, 'Score': 0.37...",0.017507,0.373239,0.003799,3827.0,132.0,65.0
2021-04-06,"Daily Discussion Thread for April 06, 2021",ml86d3,https://www.reddit.com/r/wallstreetbets/commen...,[['First you only buy calls. Then you work in ...,😔🚀🚀🙏🍏😏😡🐖⬆🤚🏼🚀🎣🎣🎣🤣🥱🤡🤡🤡🤡🤡🤡🤡🤡🤡🤡🤡🤡🤡🤡🤡🦖🙄🤡🤡🤡🍑🤮🤚🍏🚀🚀🚀🚀🚀...,first buy call Then work put Then realize week...,"{'NKLA': 1, 'UWMC': 6, 'PLTR': 42, 'ASO': 98, ...","{'Positive': 34, 'Negative': 44, 'Score': -0.0...","{'Positive': 28, 'Negative': 37, 'Score': -0.0...",-0.004663,-0.098901,-0.002510,4075.0,62.0,81.0
2021-03-24,"Daily Discussion Thread for March 24, 2021",mc1zp9,https://www.reddit.com/r/wallstreetbets/commen...,"[['[deleted]'], ['Tax on unrealised capital ga...",😏😏😏😏🌮📉📉😢🔫🙄😎😎😎🚀🚀🚀🚀🚀🚀🔔🦀💪💪🚀😂🤡🤡🌧🌈🤡😑😅😭👁👄👁🐻🤡😭🤡💨🔥🤡😔📉😟...,deleted tax unrealised capital gain retarded t...,"{'PLTR': 25, 'BODY': 2, 'GME': 21, 'CAT': 5, '...","{'Positive': 43, 'Negative': 49, 'Score': -0.0...","{'Positive': 33, 'Negative': 30, 'Score': 0.03...",-0.000659,0.039474,-0.001341,4549.0,76.0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-24,"Daily Discussion Thread for July 24, 2020",hwz949,https://www.reddit.com/r/wallstreetbets/commen...,"[['It’s not about how much money you lost, it’...",🤡😩😩😉😂🤡😪😢🙋🏻♂😎🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🤡😎🤔🌝🤦♂🤦♂🤦♂😎🥺👉🏼👈🏼😂😟🥴🤡...,much money lost much fun friend made along way...,"{'MSFT': 9, 'TSLA': 25, 'NIO': 2, 'AMD': 20, '...","{'Positive': 49, 'Negative': 58, 'Score': -0.0...","{'Positive': 43, 'Negative': 23, 'Score': 0.17...",0.002373,0.178571,-0.001990,4635.0,92.0,81.0
2020-06-10,"Daily Discussion Thread for June 10, 2020",h07z0d,https://www.reddit.com/r/wallstreetbets/commen...,"[[""When Cramer starts the fear baiting he's ta...",😭😭😭🦁😂😂🤡🤡🤡🤡🤡🤡🤡🤡🤡👈😎👉📈🚀📷😭🐂🚀🚀🚀🚀🤑🤑🥵🥵💎🤲😎🤚💎😭🚀👸😍😱🌊😤😔👻🌹...,Cramer start fear Baiting talking small retail...,"{'BALL': 3, 'MGM': 32, 'GME': 1, 'DGLY': 2, 'A...","{'Positive': 97, 'Negative': 36, 'Score': 0.01...","{'Positive': 46, 'Negative': 29, 'Score': 0.14...",0.017337,0.141667,0.013930,4499.0,143.0,65.0
2020-07-28,"Daily Discussion Thread for July 28, 2020",hzc3ko,https://www.reddit.com/r/wallstreetbets/commen...,[['Who the fuck wakes up in the morning and th...,☹😩😂😎👌🤡🤡🤡🤡🚀🚀🚀🚀🚀🤡🤡🤡😂🤡💰🐢🤙😏🤙🟢🔴🔴🔴🔴✌🏻🖕❤💎🚀😂🚀👍😂😂😂🎢😂😂😂🧻...,fuck wake morning think Im gonna award Automod...,"{'T': 2, 'LE': 4, 'SLV': 67, 'DOCU': 1, 'TSLA'...","{'Positive': 46, 'Negative': 39, 'Score': 0.00...","{'Positive': 32, 'Negative': 11, 'Score': 0.36...",0.006768,0.362069,0.001716,4137.0,78.0,50.0


In [33]:
sent = df_discussion.loc[:,['Total_sent', 'Emoji_sent', 'Text_sent']] # creating the sent table
#sent_q3_2020 = sent[sent.index>='2020-06-05']
#sent_q3_2020 = sent_q3_2020[sent_q3_2020.index<='2020-09-30']
#sent_q4_2020 = sent[sent.index>='2020-10-01']
#sent_q4_2020 = sent_q4_2020[sent_q4_2020.index<='2020-12-31']
#sent_q1_2021 = sent[sent.index>='2021-01-01']

In [34]:
#stock returns (do manually)
PLTR_returns = quandl.get("EOD/PLTR", start_date='2020-06-08', end_date='2021-04-01')['Adj_Close'].pct_change()
GME_returns = quandl.get("EOD/GME", start_date='2020-06-08', end_date='2021-04-01')['Adj_Close'].pct_change()
AMC_returns = quandl.get("EOD/AMC", start_date='2020-06-08', end_date='2021-04-01')['Adj_Close'].pct_change()
BB_returns = quandl.get("EOD/BB", start_date='2020-06-08', end_date='2021-04-01')['Adj_Close'].pct_change()
TSLA_returns = quandl.get("EOD/TSLA", start_date='2020-06-08', end_date='2021-04-01')['Adj_Close'].pct_change()
rf = quandl.get("EOD/SHV", start_date='2020-06-08', end_date='2021-04-01')['Adj_Close'].pct_change()

In [35]:
#creating df
GME = pd.DataFrame(GME_returns)
PLTR = pd.DataFrame(PLTR_returns) # PLTR went public on sept 30 2020, therefore data starts sept 30 2020
AMC = pd.DataFrame(AMC_returns)
BB = pd.DataFrame(BB_returns)
TSLA = pd.DataFrame(TSLA_returns)

# Regression Analysis

In [36]:
# Resetting index so we can merge on date
sent_1 = sent.reset_index()
GME = GME.reset_index()
PLTR = PLTR.reset_index()
AMC = AMC.reset_index()
BB = BB.reset_index()
TSLA = TSLA.reset_index()

In [37]:
# Merging all returns to the sent_1 table & renaming columns
sent_1 = pd.merge(sent_1, GME, on = 'Date', how = 'inner')
sent_1 = sent_1.rename(columns = {'Adj_Close':'GME_returns'})
sent_1 = pd.merge(sent_1, PLTR, on = 'Date', how = 'left')
sent_1 = sent_1.rename(columns = {'Adj_Close':'PLTR_returns'})
sent_1 = pd.merge(sent_1, AMC, on = 'Date', how = 'inner')
sent_1 = sent_1.rename(columns = {'Adj_Close':'AMC_returns'})
sent_1 = pd.merge(sent_1, BB, on = 'Date', how = 'inner')
sent_1 = sent_1.rename(columns = {'Adj_Close':'BB_returns'})
sent_1 = pd.merge(sent_1, TSLA, on = 'Date', how = 'inner')
sent_1 = sent_1.rename(columns = {'Adj_Close':'TSLA_returns'})


# pull the SPY data from Quandl 
SPY_returns = quandl.get("EOD/SPY")['Adj_Close'].pct_change()
SPY = pd.DataFrame(SPY_returns)
# add the SPY returns to the DF
sent_1 = pd.merge(sent_1,SPY, on = 'Date', how = 'inner')
sent_1 = sent_1.rename(columns = {'Adj_Close':'SPY_returns'})

# rf 
#sent_1['Risk_Free'] = rf

In [38]:
# find the excess market returns
sent_1['GME_exmkt_returns'] =  sent_1['GME_returns']- sent_1['SPY_returns'] 
sent_1['PLTR_exmkt_returns'] =  sent_1['PLTR_returns']- sent_1['SPY_returns'] 
sent_1['AMC_exmkt_returns'] =  sent_1['AMC_returns']- sent_1['SPY_returns'] 
sent_1['BB_exmkt_returns'] =  sent_1['BB_returns']- sent_1['SPY_returns'] 
sent_1['TSLA_exmkt_returns'] =  sent_1['TSLA_returns']- sent_1['SPY_returns'] 

In [39]:
# Lag returns 
sent_1['GME_lag_returns'] = sent_1['GME_returns'].shift()
sent_1['PLTR_lag_returns'] = sent_1['PLTR_returns'].shift()
sent_1['AMC_lag_returns'] = sent_1['AMC_returns'].shift()
sent_1['BB_lag_returns'] = sent_1['BB_returns'].shift()
sent_1['TSLA_lag_returns'] = sent_1['TSLA_returns'].shift()

In [40]:
sent_1 = sent_1.set_index('Date') # setting date as index
sent_1 = sent_1.sort_index() # Sorting index

In [49]:
sent_1

,Total_sent,Emoji_sent,Text_sent,GME_returns,PLTR_returns,AMC_returns,BB_returns,TSLA_returns,SPY_returns,GME_exmkt_returns,PLTR_exmkt_returns,AMC_exmkt_returns,BB_exmkt_returns,TSLA_exmkt_returns,GME_lag_returns,PLTR_lag_returns,AMC_lag_returns,BB_lag_returns,TSLA_lag_returns
Date,,,,,,,,,,,,,,,,,,,
2020-06-10,0.017337,0.141667,0.013930,0.022177,NaN,0.050083,-0.007130,0.089702,-0.005580,0.027757,NaN,0.055663,-0.001550,0.095282,-0.019465,NaN,-0.014778,-0.018987,-0.063493
2020-06-11,0.003145,0.018868,0.002761,-0.138067,NaN,-0.178060,-0.096948,-0.050934,-0.057649,-0.080418,NaN,-0.120412,-0.039299,0.006715,-0.004773,NaN,-0.026667,-0.012739,-0.029347
2020-06-12,0.008497,0.241379,0.005171,0.080092,NaN,0.139265,0.043738,-0.038609,0.011976,0.068116,NaN,0.127289,0.031762,-0.050584,-0.017456,NaN,0.072351,-0.006397,-0.040991
2020-06-15,-0.002913,0.257143,-0.006757,-0.006356,NaN,-0.015280,-0.049524,0.059469,0.009336,-0.015692,NaN,-0.024616,-0.058859,0.050133,0.080092,NaN,0.139265,0.043738,-0.038609
2020-07-09,0.004904,0.282051,0.000415,-0.011737,NaN,0.031603,-0.006198,0.020792,-0.005693,-0.006044,NaN,0.037296,-0.000505,0.026485,0.024938,NaN,-0.028916,-0.012371,0.015283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-30,-0.001906,0.319444,-0.007513,0.072587,0.014266,0.000000,0.015217,0.039801,-0.002653,0.075240,0.016919,0.002653,0.017870,0.042454,0.008587,-0.009446,-0.083252,0.020166,-0.009252
2021-03-31,0.001296,0.122449,-0.002677,-0.023861,0.056715,-0.013527,-0.097430,0.050832,0.004053,-0.027914,0.052662,-0.017580,-0.101484,0.046779,-0.023861,0.056715,-0.013527,-0.097430,0.050832
2021-03-31,0.010054,0.444444,-0.002529,-0.023861,0.056715,-0.013527,-0.097430,0.050832,0.004053,-0.027914,0.052662,-0.017580,-0.101484,0.046779,0.072587,0.014266,0.000000,0.015217,0.039801


In [48]:
sent_1.describe() # summary stats

,Total_sent,Emoji_sent,Text_sent,GME_returns,PLTR_returns,AMC_returns,BB_returns,TSLA_returns,SPY_returns,GME_exmkt_returns,PLTR_exmkt_returns,AMC_exmkt_returns,BB_exmkt_returns,TSLA_exmkt_returns,GME_lag_returns,PLTR_lag_returns,AMC_lag_returns,BB_lag_returns,TSLA_lag_returns
count,230.000000,230.000000,230.000000,230.000000,170.000000,230.000000,230.000000,230.000000,230.000000,230.000000,170.000000,230.000000,230.000000,230.000000,229.000000,170.000000,229.000000,229.000000,229.000000
mean,0.023447,0.306144,0.005067,0.045404,0.002563,0.034481,-0.002771,0.001542,0.000635,0.044769,0.001714,0.033846,-0.003405,0.000907,0.045630,0.002563,0.034698,-0.002566,0.001289
std,0.035495,0.294272,0.008248,0.235251,0.061612,0.316817,0.082152,0.048907,0.011259,0.237735,0.061271,0.319591,0.082077,0.044480,0.235741,0.061612,0.317494,0.082273,0.048863
min,-0.011667,-0.314286,-0.007513,-0.600000,-0.127546,-0.566332,-0.416335,-0.210628,-0.057649,-0.614140,-0.126680,-0.574932,-0.424935,-0.183305,-0.600000,-0.127546,-0.566332,-0.416335,-0.210628
25%,0.003968,0.086508,0.000207,-0.032361,-0.036356,-0.044277,-0.026217,-0.028881,-0.005083,-0.031081,-0.036544,-0.042857,-0.023269,-0.026384,-0.032544,-0.036356,-0.044643,-0.026217,-0.029347
50%,0.008723,0.266450,0.003347,0.006971,-0.008388,-0.007076,-0.003656,-0.000918,0.001387,0.003555,-0.009747,-0.008887,-0.006989,-0.001908,0.006986,-0.008388,-0.006944,-0.003656,-0.001004
75%,0.028950,0.541964,0.007584,0.057345,0.031993,0.043655,0.020569,0.027975,0.007985,0.059868,0.026367,0.043064,0.019297,0.022245,0.057447,0.031993,0.043655,0.020704,0.027813
max,0.201417,0.931129,0.050500,1.348358,0.219563,3.012097,0.326638,0.196412,0.024240,1.372798,0.221105,3.036537,0.351079,0.182135,1.348358,0.219563,3.012097,0.326638,0.196412


# Total Sent analysis with lag

## GME reg

In [51]:
# Run an OLS regression 
reg = smf.ols('GME_exmkt_returns~Total_sent+GME_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      GME_exmkt_returns   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                     67.96
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           7.76e-24
Time:                        14:22:19   Log-Likelihood:                 59.237
No. Observations:                 229   AIC:                            -112.5
Df Residuals:                     226   BIC:                            -102.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0020      0.015     -0.133      0.894      -0.031       0.027
Total_sent          1.0014      0.377      2.654      0.009       0.258       1.745
GME_lag_returns     0.5439      0.057      9.562      0.000       0.432       0.656
==============================================================================
Omnibus:                       51.134   Durbin-Watson:                   1.430
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              800.463
Skew:                          -0.182   Prob(JB):                    1.52e-174
Kurtosis:                      12.152   Cond. No.                         30.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## PLTR reg

In [52]:
# Run an OLS regression 
reg = smf.ols('PLTR_exmkt_returns~Total_sent+PLTR_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     PLTR_exmkt_returns   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     3.803
Date:                Tue, 13 Apr 2021   Prob (F-statistic):             0.0243
Time:                        14:22:27   Log-Likelihood:                 229.18
No. Observations:                 165   AIC:                            -452.4
Df Residuals:                     162   BIC:                            -443.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.0028      0.006     -0.470      0.639      -0.015       0.009
Total_sent           0.1595      0.120      1.328      0.186      -0.078       0.397
PLTR_lag_returns     0.1874      0.076      2.460      0.015       0.037       0.338
==============================================================================
Omnibus:                       23.224   Durbin-Watson:                   1.546
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               36.179
Skew:                           0.749   Prob(JB):                     1.39e-08
Kurtosis:                       4.737   Cond. No.                         25.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## AMC reg

In [53]:
# Run an OLS regression 
reg = smf.ols('AMC_exmkt_returns~Total_sent+AMC_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      AMC_exmkt_returns   R-squared:                       0.351
Model:                            OLS   Adj. R-squared:                  0.345
Method:                 Least Squares   F-statistic:                     61.11
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           6.10e-22
Time:                        14:22:28   Log-Likelihood:                -14.052
No. Observations:                 229   AIC:                             34.10
Df Residuals:                     226   BIC:                             44.41
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0421      0.021     -2.037      0.043      -0.083      -0.001
Total_sent          2.6332      0.517      5.098      0.000       1.615       3.651
AMC_lag_returns     0.4250      0.058      7.351      0.000       0.311       0.539
==============================================================================
Omnibus:                      249.152   Durbin-Watson:                   1.550
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            26345.148
Skew:                           3.915   Prob(JB):                         0.00
Kurtosis:                      54.959   Cond. No.                         30.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## BB reg

In [54]:
# Run an OLS regression 
reg = smf.ols('BB_exmkt_returns~Total_sent+BB_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       BB_exmkt_returns   R-squared:                       0.349
Model:                            OLS   Adj. R-squared:                  0.343
Method:                 Least Squares   F-statistic:                     60.45
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           9.37e-22
Time:                        14:22:29   Log-Likelihood:                 296.77
No. Observations:                 229   AIC:                            -587.5
Df Residuals:                     226   BIC:                            -577.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.0098      0.005     -1.854      0.065      -0.020       0.001
Total_sent         0.3413      0.124      2.745      0.007       0.096       0.586
BB_lag_returns     0.5647      0.054     10.515      0.000       0.459       0.671
==============================================================================
Omnibus:                       85.022   Durbin-Watson:                   1.802
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1122.711
Skew:                          -1.027   Prob(JB):                    1.61e-244
Kurtosis:                      13.651   Cond. No.                         28.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## TSLA

In [55]:
# Run an OLS regression 
reg = smf.ols('TSLA_exmkt_returns~Total_sent+TSLA_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     TSLA_exmkt_returns   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     5.910
Date:                Tue, 13 Apr 2021   Prob (F-statistic):            0.00315
Time:                        14:22:30   Log-Likelihood:                 394.21
No. Observations:                 229   AIC:                            -782.4
Df Residuals:                     226   BIC:                            -772.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0005      0.003      0.147      0.884      -0.006       0.007
Total_sent           0.0140      0.082      0.171      0.864      -0.147       0.175
TSLA_lag_returns     0.2038      0.059      3.432      0.001       0.087       0.321
==============================================================================
Omnibus:                       25.048   Durbin-Watson:                   1.816
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               68.507
Skew:                           0.432   Prob(JB):                     1.33e-15
Kurtosis:                       5.536   Cond. No.                         28.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Total Sent analysis without lag

# GME

In [56]:
# Run an OLS regression 
reg = smf.ols('GME_exmkt_returns~Total_sent', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      GME_exmkt_returns   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     32.47
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           3.72e-08
Time:                        14:22:34   Log-Likelihood:                 19.873
No. Observations:                 230   AIC:                            -35.75
Df Residuals:                     228   BIC:                            -28.87
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0107      0.018     -0.606      0.545      -0.045       0.024
Total_sent     2.3647      0.415      5.698      0.000       1.547       3.182
==============================================================================
Omnibus:                       74.728   Durbin-Watson:                   1.213
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              497.528
Skew:                           1.077   Prob(JB):                    9.19e-109
Kurtosis:                       9.876   Cond. No.                         28.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# PLTR

In [57]:
# Run an OLS regression 
reg = smf.ols('PLTR_exmkt_returns~Total_sent', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     PLTR_exmkt_returns   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.872
Date:                Tue, 13 Apr 2021   Prob (F-statistic):              0.173
Time:                        14:22:35   Log-Likelihood:                 234.94
No. Observations:                 170   AIC:                            -465.9
Df Residuals:                     168   BIC:                            -459.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0031      0.006     -0.530      0.597      -0.015       0.008
Total_sent     0.1635      0.120      1.368      0.173      -0.072       0.399
==============================================================================
Omnibus:                       27.411   Durbin-Watson:                   1.484
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               39.260
Skew:                           0.904   Prob(JB):                     2.98e-09
Kurtosis:                       4.507   Cond. No.                         25.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# AMC reg

In [58]:
# Run an OLS regression 
reg = smf.ols('AMC_exmkt_returns~Total_sent', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      AMC_exmkt_returns   R-squared:                       0.197
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     55.81
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           1.69e-12
Time:                        14:22:35   Log-Likelihood:                -38.311
No. Observations:                 230   AIC:                             80.62
Df Residuals:                     228   BIC:                             87.50
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0598      0.023     -2.633      0.009      -0.105      -0.015
Total_sent     3.9927      0.534      7.471      0.000       2.940       5.046
==============================================================================
Omnibus:                      268.175   Durbin-Watson:                   1.387
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17874.473
Skew:                           4.678   Prob(JB):                         0.00
Kurtosis:                      45.162   Cond. No.                         28.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# BB

In [59]:
# Run an OLS regression 
reg = smf.ols('BB_exmkt_returns~Total_sent', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       BB_exmkt_returns   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     7.093
Date:                Tue, 13 Apr 2021   Prob (F-statistic):            0.00829
Time:                        14:22:37   Log-Likelihood:                 252.69
No. Observations:                 230   AIC:                            -501.4
Df Residuals:                     228   BIC:                            -494.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0128      0.006     -2.002      0.047      -0.025      -0.000
Total_sent     0.4017      0.151      2.663      0.008       0.104       0.699
==============================================================================
Omnibus:                      148.994   Durbin-Watson:                   1.053
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2493.916
Skew:                          -2.186   Prob(JB):                         0.00
Kurtosis:                      18.528   Cond. No.                         28.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# TSLA

In [60]:
# Run an OLS regression 
reg = smf.ols('TSLA_exmkt_returns~Total_sent', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     TSLA_exmkt_returns   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                   0.02255
Date:                Tue, 13 Apr 2021   Prob (F-statistic):              0.881
Time:                        14:22:38   Log-Likelihood:                 390.08
No. Observations:                 230   AIC:                            -776.2
Df Residuals:                     228   BIC:                            -769.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0012      0.004      0.340      0.734      -0.006       0.008
Total_sent    -0.0125      0.083     -0.150      0.881      -0.176       0.151
==============================================================================
Omnibus:                       34.091   Durbin-Watson:                   1.680
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.438
Skew:                           0.634   Prob(JB):                     1.39e-20
Kurtosis:                       5.817   Cond. No.                         28.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Emoji Sent analysis

# GME

In [61]:
# Run an OLS regression 
reg = smf.ols('GME_exmkt_returns~Emoji_sent+GME_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      GME_exmkt_returns   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     64.32
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           7.74e-23
Time:                        14:22:39   Log-Likelihood:                 56.907
No. Observations:                 229   AIC:                            -107.8
Df Residuals:                     226   BIC:                            -97.51
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0007      0.018     -0.040      0.968      -0.037       0.035
Emoji_sent          0.0668      0.044      1.533      0.127      -0.019       0.153
GME_lag_returns     0.5833      0.054     10.713      0.000       0.476       0.691
==============================================================================
Omnibus:                       49.235   Durbin-Watson:                   1.441
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              742.652
Skew:                           0.113   Prob(JB):                    5.43e-162
Kurtosis:                      11.819   Cond. No.                         4.71
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# PLTR

In [62]:
# Run an OLS regression 
reg = smf.ols('PLTR_exmkt_returns~Emoji_sent+PLTR_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     PLTR_exmkt_returns   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     2.934
Date:                Tue, 13 Apr 2021   Prob (F-statistic):             0.0560
Time:                        14:22:40   Log-Likelihood:                 228.33
No. Observations:                 165   AIC:                            -450.7
Df Residuals:                     162   BIC:                            -441.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0003      0.007      0.046      0.964      -0.014       0.015
Emoji_sent           0.0046      0.016      0.295      0.768      -0.026       0.035
PLTR_lag_returns     0.1844      0.077      2.409      0.017       0.033       0.336
==============================================================================
Omnibus:                       22.550   Durbin-Watson:                   1.558
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               33.361
Skew:                           0.753   Prob(JB):                     5.70e-08
Kurtosis:                       4.607   Cond. No.                         17.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# AMC

In [63]:
# Run an OLS regression 
reg = smf.ols('AMC_exmkt_returns~Emoji_sent+AMC_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      AMC_exmkt_returns   R-squared:                       0.289
Model:                            OLS   Adj. R-squared:                  0.283
Method:                 Least Squares   F-statistic:                     45.96
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           1.79e-17
Time:                        14:22:41   Log-Likelihood:                -24.476
No. Observations:                 229   AIC:                             54.95
Df Residuals:                     226   BIC:                             65.25
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0214      0.026     -0.826      0.410      -0.073       0.030
Emoji_sent          0.1252      0.062      2.015      0.045       0.003       0.248
AMC_lag_returns     0.5078      0.058      8.817      0.000       0.394       0.621
==============================================================================
Omnibus:                      282.575   Durbin-Watson:                   1.434
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            36133.640
Skew:                           4.850   Prob(JB):                         0.00
Kurtosis:                      63.769   Cond. No.                         3.98
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# BB

In [64]:
# Run an OLS regression 
reg = smf.ols('BB_exmkt_returns~Emoji_sent+BB_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       BB_exmkt_returns   R-squared:                       0.327
Model:                            OLS   Adj. R-squared:                  0.321
Method:                 Least Squares   F-statistic:                     54.95
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           3.58e-20
Time:                        14:22:42   Log-Likelihood:                 293.08
No. Observations:                 229   AIC:                            -580.2
Df Residuals:                     226   BIC:                            -569.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.0034      0.006     -0.529      0.598      -0.016       0.009
Emoji_sent         0.0054      0.015      0.355      0.723      -0.025       0.035
BB_lag_returns     0.5714      0.055     10.480      0.000       0.464       0.679
==============================================================================
Omnibus:                       53.524   Durbin-Watson:                   1.813
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              975.815
Skew:                          -0.135   Prob(JB):                    1.27e-212
Kurtosis:                      13.109   Cond. No.                         12.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# TSLA

In [65]:
# Run an OLS regression 
reg = smf.ols('TSLA_exmkt_returns~Emoji_sent+TSLA_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     TSLA_exmkt_returns   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     6.449
Date:                Tue, 13 Apr 2021   Prob (F-statistic):            0.00189
Time:                        14:22:43   Log-Likelihood:                 394.72
No. Observations:                 229   AIC:                            -783.4
Df Residuals:                     226   BIC:                            -773.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.0022      0.004     -0.540      0.590      -0.010       0.006
Emoji_sent           0.0100      0.010      1.027      0.306      -0.009       0.029
TSLA_lag_returns     0.2022      0.059      3.432      0.001       0.086       0.318
==============================================================================
Omnibus:                       23.523   Durbin-Watson:                   1.802
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               59.250
Skew:                           0.429   Prob(JB):                     1.36e-13
Kurtosis:                       5.340   Cond. No.                         21.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Text Sent analysis

# GME

In [66]:
# Run an OLS regression 
reg = smf.ols('GME_exmkt_returns~Text_sent+GME_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      GME_exmkt_returns   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                  0.363
Method:                 Least Squares   F-statistic:                     65.99
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           2.68e-23
Time:                        14:22:47   Log-Likelihood:                 57.982
No. Observations:                 229   AIC:                            -110.0
Df Residuals:                     226   BIC:                            -99.66
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0035      0.015      0.236      0.813      -0.025       0.032
Text_sent           3.3704      1.588      2.122      0.035       0.241       6.500
GME_lag_returns     0.5648      0.056     10.154      0.000       0.455       0.674
==============================================================================
Omnibus:                       50.916   Durbin-Watson:                   1.430
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              846.942
Skew:                          -0.077   Prob(JB):                    1.23e-184
Kurtosis:                      12.420   Cond. No.                         127.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# PLTR

In [67]:
# Run an OLS regression 
reg = smf.ols('PLTR_exmkt_returns~Text_sent+PLTR_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     PLTR_exmkt_returns   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     3.074
Date:                Tue, 13 Apr 2021   Prob (F-statistic):             0.0489
Time:                        14:22:47   Log-Likelihood:                 228.47
No. Observations:                 165   AIC:                            -450.9
Df Residuals:                     162   BIC:                            -441.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         7.921e-05      0.006      0.014      0.989      -0.011       0.011
Text_sent            0.3123      0.522      0.598      0.551      -0.719       1.344
PLTR_lag_returns     0.1876      0.077      2.446      0.016       0.036       0.339
==============================================================================
Omnibus:                       22.825   Durbin-Watson:                   1.556
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               34.028
Skew:                           0.758   Prob(JB):                     4.08e-08
Kurtosis:                       4.628   Cond. No.                         110.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# AMC

In [68]:
# Run an OLS regression 
reg = smf.ols('AMC_exmkt_returns~Text_sent+AMC_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      AMC_exmkt_returns   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.305
Method:                 Least Squares   F-statistic:                     50.96
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           5.42e-19
Time:                        14:22:49   Log-Likelihood:                -20.932
No. Observations:                 229   AIC:                             47.86
Df Residuals:                     226   BIC:                             58.16
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0202      0.021     -0.970      0.333      -0.061       0.021
Text_sent           7.5560      2.248      3.361      0.001       3.125      11.987
AMC_lag_returns     0.4700      0.058      8.039      0.000       0.355       0.585
==============================================================================
Omnibus:                      273.650   Durbin-Watson:                   1.479
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33812.896
Skew:                           4.581   Prob(JB):                         0.00
Kurtosis:                      61.820   Cond. No.                         128.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# BB

In [69]:
# Run an OLS regression 
reg = smf.ols('BB_exmkt_returns~Text_sent+BB_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       BB_exmkt_returns   R-squared:                       0.331
Model:                            OLS   Adj. R-squared:                  0.325
Method:                 Least Squares   F-statistic:                     55.83
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           1.98e-20
Time:                        14:22:50   Log-Likelihood:                 293.68
No. Observations:                 229   AIC:                            -581.4
Df Residuals:                     226   BIC:                            -571.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.0049      0.005     -0.939      0.349      -0.015       0.005
Text_sent          0.6231      0.543      1.147      0.252      -0.447       1.693
BB_lag_returns     0.5751      0.054     10.556      0.000       0.468       0.683
==============================================================================
Omnibus:                       61.687   Durbin-Watson:                   1.808
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1093.616
Skew:                          -0.441   Prob(JB):                    3.34e-238
Kurtosis:                      13.669   Cond. No.                         122.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# TSLA

In [70]:
# Run an OLS regression 
reg = smf.ols('TSLA_exmkt_returns~Text_sent+TSLA_lag_returns', data=sent_1).fit()
# look at the regression output
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     TSLA_exmkt_returns   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     5.895
Date:                Tue, 13 Apr 2021   Prob (F-statistic):            0.00319
Time:                        14:22:52   Log-Likelihood:                 394.19
No. Observations:                 229   AIC:                            -782.4
Df Residuals:                     226   BIC:                            -772.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0009      0.003      0.266      0.791      -0.006       0.008
Text_sent           -0.0132      0.355     -0.037      0.970      -0.712       0.686
TSLA_lag_returns     0.2023      0.060      3.374      0.001       0.084       0.320
==============================================================================
Omnibus:                       25.170   Durbin-Watson:                   1.819
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               69.690
Skew:                           0.430   Prob(JB):                     7.36e-16
Kurtosis:                       5.562   Cond. No.                         123.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Cross - section analysis

In [73]:
# Adding all stock excess returns along with their total sent scores into a df
gme =sent_1.loc[:,['GME_exmkt_returns', 'Total_sent']]
gme = gme.rename(columns = {'GME_exmkt_returns':'Excess_Returns'})
pltr = sent_1.loc[:,['PLTR_exmkt_returns', 'Total_sent']]
pltr = pltr.rename(columns = {'PLTR_exmkt_returns':'Excess_Returns'})
x = pd.concat([gme,pltr])
amc =sent_1.loc[:,['AMC_exmkt_returns','Total_sent']]
amc = amc.rename(columns = {'AMC_exmkt_returns':'Excess_Returns'})
x1 = pd.concat([x,amc])
bb = sent_1.loc[:,['BB_exmkt_returns', 'Total_sent']]
bb = bb.rename(columns = {'BB_exmkt_returns':'Excess_Returns'})
x2 = pd.concat([x1,bb])
tsla = sent_1.loc[:,['TSLA_exmkt_returns','Total_sent']]
tsla = tsla.rename(columns = {'TSLA_exmkt_returns':'Excess_Returns'})
x3 = pd.concat([x2,tsla])
x3 = x3.sort_index()
x3

,Excess_Returns,Total_sent
Date,,
2020-06-10,0.027757,0.017337
2020-06-10,0.055663,0.017337
2020-06-10,-0.001550,0.017337
2020-06-10,NaN,0.017337
2020-06-10,0.095282,0.017337
...,...,...
2021-04-01,-0.094051,0.004487
2021-04-01,-0.094051,-0.005221
2021-04-01,-0.020052,-0.005221


In [74]:
reg = smf.ols('Excess_Returns~Total_sent', data=x3).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Excess_Returns   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     81.43
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           8.04e-19
Time:                        14:32:35   Log-Likelihood:                 302.19
No. Observations:                1090   AIC:                            -600.4
Df Residuals:                    1088   BIC:                            -590.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0176      0.007     -2.617      0.009      -0.031      -0.004
Total_sent     1.3896      0.154      9.024      0.000       1.087       1.692
==============================================================================
Omnibus:                     1489.368   Durbin-Watson:                   1.517
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           515383.829
Skew:                           7.231   Prob(JB):                         0.00
Kurtosis:                     108.540   Cond. No.                         27.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [90]:
# Adding all stock excess returns along with their emoji sent scores into a df
gme =sent_1.loc[:,['GME_exmkt_returns', 'Emoji_sent']]
gme = gme.rename(columns = {'GME_exmkt_returns':'Excess_Returns'})
pltr = sent_1.loc[:,['PLTR_exmkt_returns', 'Emoji_sent']]
pltr = pltr.rename(columns = {'PLTR_exmkt_returns':'Excess_Returns'})
x = pd.concat([gme,pltr])
amc =sent_1.loc[:,['AMC_exmkt_returns','Emoji_sent']]
amc = amc.rename(columns = {'AMC_exmkt_returns':'Excess_Returns'})
x1 = pd.concat([x,amc])
bb = sent_1.loc[:,['BB_exmkt_returns', 'Emoji_sent']]
bb = bb.rename(columns = {'BB_exmkt_returns':'Excess_Returns'})
x2 = pd.concat([x1,bb])
tsla = sent_1.loc[:,['TSLA_exmkt_returns','Emoji_sent']]
tsla = tsla.rename(columns = {'TSLA_exmkt_returns':'Excess_Returns'})
x3 = pd.concat([x2,tsla])
x3 = x3.sort_index()
x3

,Excess_Returns,Emoji_sent
Date,,
2020-06-10,0.027757,0.141667
2020-06-10,0.055663,0.141667
2020-06-10,-0.001550,0.141667
2020-06-10,NaN,0.141667
2020-06-10,0.095282,0.141667
...,...,...
2021-04-01,-0.094051,0.281250
2021-04-01,-0.094051,-0.016667
2021-04-01,-0.020052,-0.016667


In [91]:
reg = smf.ols('Excess_Returns~Emoji_sent', data=x3).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Excess_Returns   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     19.30
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           1.23e-05
Time:                        14:53:27   Log-Likelihood:                 272.43
No. Observations:                1090   AIC:                            -540.9
Df Residuals:                    1088   BIC:                            -530.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0103      0.008     -1.234      0.218      -0.027       0.006
Emoji_sent     0.0846      0.019      4.393      0.000       0.047       0.122
==============================================================================
Omnibus:                     1584.327   Durbin-Watson:                   1.439
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           646227.481
Skew:                           8.123   Prob(JB):                         0.00
Kurtosis:                     121.173   Cond. No.                         3.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [92]:
# Adding all stock excess returns along with their text sent scores into a df
gme =sent_1.loc[:,['GME_exmkt_returns', 'Text_sent']]
gme = gme.rename(columns = {'GME_exmkt_returns':'Excess_Returns'})
pltr = sent_1.loc[:,['PLTR_exmkt_returns', 'Text_sent']]
pltr = pltr.rename(columns = {'PLTR_exmkt_returns':'Excess_Returns'})
x = pd.concat([gme,pltr])
amc =sent_1.loc[:,['AMC_exmkt_returns','Text_sent']]
amc = amc.rename(columns = {'AMC_exmkt_returns':'Excess_Returns'})
x1 = pd.concat([x,amc])
bb = sent_1.loc[:,['BB_exmkt_returns', 'Text_sent']]
bb = bb.rename(columns = {'BB_exmkt_returns':'Excess_Returns'})
x2 = pd.concat([x1,bb])
tsla = sent_1.loc[:,['TSLA_exmkt_returns','Text_sent']]
tsla = tsla.rename(columns = {'TSLA_exmkt_returns':'Excess_Returns'})
x3 = pd.concat([x2,tsla])
x3 = x3.sort_index()
x3

,Excess_Returns,Text_sent
Date,,
2020-06-10,0.027757,0.013930
2020-06-10,0.055663,0.013930
2020-06-10,-0.001550,0.013930
2020-06-10,NaN,0.013930
2020-06-10,0.095282,0.013930
...,...,...
2021-04-01,-0.094051,-0.004140
2021-04-01,-0.094051,-0.004783
2021-04-01,-0.020052,-0.004783


In [93]:
reg = smf.ols('Excess_Returns~Text_sent', data=x3).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Excess_Returns   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     42.37
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           1.15e-10
Time:                        14:54:58   Log-Likelihood:                 283.67
No. Observations:                1090   AIC:                            -563.3
Df Residuals:                    1088   BIC:                            -553.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0066      0.007     -0.994      0.320      -0.020       0.006
Text_sent      4.3961      0.675      6.509      0.000       3.071       5.721
==============================================================================
Omnibus:                     1540.589   Durbin-Watson:                   1.461
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           591958.378
Skew:                           7.695   Prob(JB):                         0.00
Kurtosis:                     116.124   Cond. No.                         119.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""